# Correr esta celda al iniciar

In [ ]:
import sys
import io
import os
import re

import cv2
import numpy as np
from pyzbar.pyzbar import decode
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
from matplotlib import pyplot as plt

sys.path.insert(0, '../QR Query/Query QRs/Scripts')
sys.path.insert(0, '../FIND/Golden Master (AS IS)')
import qr_query
import read_image as ri
from Browser import Browser
from DataBrowser import DataBrowser


# Elige cómo hacer la búsqueda

In [ ]:
folder = './qrsSubidos/'
if not os.path.isdir(folder):
    os.mkdir(folder)
option = widgets.ToggleButtons(
    options=['Escribir QR', 'Seleccionar archivo', 'Rango QR'],
    description='Búsqueda:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)
display(option)
print('Después de seleccionar, corre la siguiente celda')

# Escribe QRs o sube imágenes

In [ ]:
if option.value == 'Escribir QR':
    textQrs = widgets.Text(
        value='',
        placeholder='Escribe un código válido de QR',
        description='Código QR:',
        disabled=False
    )
    display(textQrs)
    print('Puedes introducir varios separando con espacios o comas')
elif option.value == 'Seleccionar archivo':
    uploads = widgets.FileUpload(
        accept='.jpg',
        multiple=True, # True to accept multiple files upload else False
        description='Imagen'
    )
    display(uploads)
elif option.value == 'Rango QR':
    textRange = widgets.Text(
        value='',
        placeholder='Código de inicio y de final',
        description='Rangos QR:',
        disabled=False
    )
    display(textRange)
print('Luego de terminar, corre la siguiente celda')

# Comenzar búsqueda

In [ ]:
csvName = 'Query.csv'
csvFolder = 'Hojas de datos'
qrList = []
if not os.path.isdir(csvFolder):
    os.mkdir(csvFolder)
csvFile = os.path.join(csvFolder, csvName)
if option.value == 'Escribir QR':
    qrList = qr_query.splitInputQrs(textQrs.value)
    if not qrList:
        print('Escribe un QR arriba')
elif option.value == 'Rango QR':
    inputList = qr_query.splitInputQrs(textRange.value)
    if inputList:
        if len(inputList) == 2:
            if inputList[0] > inputList[1]:
                start = int(inputList[1])
                finish = int(inputList[0])
            else:
                start = int(inputList[0])
                finish = int(inputList[1])
            qrList = np.arange(start, finish+1, 1)
            qrList = [str(qr) for qr in qrList]
        else:
            print('Escribe un QR de inicio y uno de final\n')
    else:
        print('El espacio de escritura está vacío')
elif option.value == 'Seleccionar archivo':
    for upload in uploads.metadata:
        fileName = upload['name']
        imageBytes = uploads.value[upload['name']]['content']
        imageBytes = io.BytesIO(imageBytes)
        image = Image.open(imageBytes)
        qr = str(decode(image)[0][0])[2:-1]
        qrList.append(qr)
    if len(qrList) == 0:
        print('Ningún QR pudo ser detectado\n')
    elif len(qrList) < len(uploads.metadata):
        print('No todos los QR pudieron detectarse\n')

for qr in qrList:
    if qr_query.isValidQr(qr):
        data = qr_query.getData(qr)
        if data is not None:
            print('Los datos completos de los QR están en el archivo {}'.format(csvFile))
            data.to_csv(csvFile, mode='a', header=False, index=False)
        else: 
            print('No existen registros del QR: {}'.format(qr))
        images = qr_query.getImages(qr)
        nImages = len(images)
        if nImages > 0:
            amount = int(input('Se encontraron {} imágenes, cuántas deseas mostrar? '.format(nImages)))
            while True:
                if amount <= 0 or amount > nImages:
                    amount = int(input('Selecciona un número entre 1 y {} '.format(nImages))) 
                else:
                    break
            ri.showImages(images[0:amount], amount)
        else:
            print('No existen imagenes del QR: {}\n'.format(qr))
    else:
        print('El QR: {} es inválido'.format(qr))
        continue
        
print('Búsqueda terminada')